In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/test.csv
/kaggle/input/fake-news/train.csv


In [2]:
train=pd.read_csv(r"/kaggle/input/fake-news/train.csv")
test=pd.read_csv(r"/kaggle/input/fake-news/test.csv")

In [3]:
train.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [4]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [5]:
###Drop Nan Values
train=train.dropna()


In [6]:
## Get the Independent Features

X=train.drop('label',axis=1)

In [7]:
## Get the Dependent features
y=train['label']

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [11]:

### Vocabulary size
voc_size=5000

## Onehot Representation

In [12]:
messages=X.copy()

In [13]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:
messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [17]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def data_preprocessing(messages):
    corpus = []
    for i in range(0, len(messages)):
    
        review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
        review = review.lower()
        review = review.split()
    
        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
        #corpus
    return [one_hot(words,voc_size)for words in corpus] 


onehot_repr=data_preprocessing(messages)

In [18]:

#onehot_repr

## Embedding Representation

In [19]:
sent_length=20
def embedding_representation(onehot_repr):
    
    embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
    return embedded_docs

embedded_docs=embedding_representation(onehot_repr)


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2902,
        868, 3442, 2841, 2473, 1748, 3465,  724, 2439,  120], dtype=int32)

## Creating model

In [21]:
from tensorflow.keras.layers import Dropout

#create model

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    


In [22]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(18285, (18285,))

In [24]:

import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:

X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [27]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 7s 583us/sample - loss: 0.3457 - accuracy: 0.8342 - val_loss: 0.2029 - val_accuracy: 0.9132
Epoch 2/10
12250/12250 [==============================] - 2s 137us/sample - loss: 0.1502 - accuracy: 0.9407 - val_loss: 0.1945 - val_accuracy: 0.9198
Epoch 3/10
12250/12250 [==============================] - 2s 140us/sample - loss: 0.1154 - accuracy: 0.9565 - val_loss: 0.2107 - val_accuracy: 0.9206
Epoch 4/10
12250/12250 [==============================] - 2s 190us/sample - loss: 0.0902 - accuracy: 0.9656 - val_loss: 0.2309 - val_accuracy: 0.9105
Epoch 5/10
12250/12250 [==============================] - 2s 148us/sample - loss: 0.0739 - accuracy: 0.9742 - val_loss: 0.2535 - val_accuracy: 0.9127
Epoch 6/10
12250/12250 [==============================] - 2s 170us/sample - loss: 0.0577 - accuracy: 0.9805 - val_loss: 0.3062 - val_accuracy: 0.9125
Epoch 7/10
12250/12250 [===========================


# Performance Metrics And Accuracy

In [29]:
y_pred=model.predict_classes(X_test)

In [30]:
from sklearn.metrics import confusion_matrix

In [31]:

confusion_matrix(y_test,y_pred)

array([[3108,  311],
       [ 238, 2378]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9090306545153273

# Test Data Prediction

In [33]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [34]:
test_messages=test.copy()
test_messages = test_messages.fillna(' ')


In [35]:
test_messages['title'].head()

0    Specter of Trump Loosens Tongues, if Not Purse...
1    Russian warships ready to strike terrorists ne...
2    #NoDAPL: Native American Leaders Vow to Stay A...
3    Tim Tebow Will Attempt Another Comeback, This ...
4                      Keiser Report: Meme Wars (E995)
Name: title, dtype: object

In [36]:
test_messages['title'].isnull().sum()

0

In [37]:
len(test_messages['title'])

5200

In [38]:
test_messages.reset_index(inplace=True)

In [39]:
test_onehot_repr=data_preprocessing(test_messages)

In [40]:
test_embedded_docs=embedding_representation(test_onehot_repr)

In [41]:
test_embedded_docs

array([[   0,    0,    0, ...,  663, 1059, 3158],
       [   0,    0,    0, ..., 3969, 4897, 3064],
       [   0,    0,    0, ...,  499,  741, 4387],
       ...,
       [   0,    0,    0, ...,  663, 1059, 3158],
       [   0,    0,    0, ..., 3742, 4136, 1110],
       [   0,    0,    0, ...,  663, 1059, 3158]], dtype=int32)

In [42]:

test_final=np.array(test_embedded_docs)

In [43]:
test_final

array([[   0,    0,    0, ...,  663, 1059, 3158],
       [   0,    0,    0, ..., 3969, 4897, 3064],
       [   0,    0,    0, ...,  499,  741, 4387],
       ...,
       [   0,    0,    0, ...,  663, 1059, 3158],
       [   0,    0,    0, ..., 3742, 4136, 1110],
       [   0,    0,    0, ...,  663, 1059, 3158]], dtype=int32)

In [44]:
model_prediction = model.predict_classes(test_final)

In [45]:
model_prediction=model_prediction.ravel()

In [46]:
model_prediction

array([0, 1, 1, ..., 0, 1, 0], dtype=int32)

In [47]:
submission = pd.DataFrame({'id':test_messages['id'], 'label':model_prediction})
submission.shape

(5200, 2)

In [48]:
submission.head()

,id,label
0,20800,0
1,20801,1
2,20802,1
3,20803,0
4,20804,1


In [49]:
submission.to_csv('submit.csv', index = False)